# 1. Dataprocessing

In [8]:
import os
mao_sentences = []
for root, dirs, files in os.walk("../mao/"):
    for file in files:
        if file.endswith(".txt"):
            print(file)
            with open(os.path.join(root, file), "r") as input:
                sentences = input.read().replace('?','.').replace('!','.').replace('\n','.')
                sentences = sentences.split(".")   #. denotes end of sentence
            mao_sentences.extend(sentences)
            
print(len(mao_sentences))
print(mao_sentences[0])

REPORT ON AN INVESTIGATION OF THE PEASANT MOVEMENT IN HUNAN.txt
ON CORRECTING MISTAKEN IDEAS IN THE PARTY.txt
A SINGLE SPARK CAN START A PRAIRIE FIRE.txt
ANALYSIS OF THE CLASSES IN CHINESE SOCIETY.txt
BE CONCERNED WITH THE WELL-BEING OF THE MASSES, PAY ATTENTION TO METHODS OF WORK.txt
WHY IS IT THAT RED POLITICAL POWER CAN EXIST IN CHINA.txt
1702
THE IMPORTANCE OF THE PEASANT PROBLEM


In [6]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

mao_df = pd.DataFrame(mao_sentences, columns = ['Sentence'])
mao_df.head()

,Sentence
0,THE IMPORTANCE OF THE PEASANT PROBLEM
1,During my recent visit to Hunan [1] I made a f...
2,In the thirty-two days from January 4 to Febr...
3,Many of the hows and whys of the peasant move...
4,I saw and heard of many strange things of whi...


In [7]:
mao_df['NumWords'] = mao_df['Sentence'].apply(lambda x: len(x.split()))

In [8]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = ''
    lower = texts.lower()
    no_punctuation = NON_ALPHANUM.sub(r' ', lower)
    no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
    return no_non_ascii
  
mao_df['Sentence2'] = mao_df['Sentence'].apply(normalize_texts)
mao_df['Sentence2'] = mao_df['Sentence2'].apply(lambda x: ' '.join([w for w in x.split()]))

In [9]:
mao_df = mao_df[mao_df.NumWords>=5]
mao_df = mao_df.reset_index(drop=True)
mao_df

,Sentence,NumWords,Sentence2
0,THE IMPORTANCE OF THE PEASANT PROBLEM,6,the importance of the peasant problem
1,During my recent visit to Hunan [1] I made a f...,25,during my recent visit to hunan 1 i made a fir...
2,In the thirty-two days from January 4 to Febr...,48,in the thirty two days from january to februar...
3,Many of the hows and whys of the peasant move...,24,many of the hows and whys of the peasant movem...
4,I saw and heard of many strange things of whi...,15,i saw and heard of many strange things of whic...
...,...,...,...
1167,THE PROBLEM OF MILITARY BASES,5,the problem of military bases
1168,"The Party in the border area has another task,...",22,the party in the border area has another task ...
1169,The Five Wells mountain area at the juncture ...,88,the five wells mountain area at the juncture o...
1170,"The way to consolidate these bases is, first,...",27,the way to consolidate these bases is first to...


# 2. Neuralcoref

In [11]:
!pip uninstall -y neuralcoref
!pip install --no-cache-dir  neuralcoref --no-binary neuralcoref
!pip install -U spacy==2.1.0
!python -m spacy download en_core_web_lg

Found existing installation: neuralcoref 4.0
Uninstalling neuralcoref-4.0:
  Successfully uninstalled neuralcoref-4.0
     |████████████████████████████████| 368 kB 5.3 MB/s 
Skipping wheel build for neuralcoref, due to binaries being disabled for it.
    Running setup.py install for neuralcoref ... done
     |████████████████████████████████| 826.9 MB 1.1 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-py3-none-any.whl size=828255080 sha256=2bf29a7146a7f12fd05a0a3f9427c5784e2949afd937cf53d6c28934721fdbf4
  Stored in directory: /tmp/pip-ephem-wheel-cache-0x_yn6na/wheels/83/c8/3b/6640ce3755cb98381fbf391ba5fc279eac813321c316d8c7ee
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [12]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [13]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

100%|██████████| 40155833/40155833 [00:00<00:00, 40865758.32B/s]


In [14]:
text = "Eva and Martha didn't want their friend Jenny to feel lonely so they invited her to the party in Las Vegas."
doc = nlp(text)

In [15]:
doc._.coref_resolved

"Eva and Martha didn't want Eva and Martha friend Jenny to feel lonely so Eva and Martha invited Jenny to the party in Las Vegas."

Using the neuralcof on Mao's article. I record the sentences if the neuralcof do some changes on it

In [26]:
count=0            #calculate how many sentences modified by neuralcoref
original_sentence = []      
changed_sentence = []
def coreferenceResolution(sentence):
  global count
  doc = nlp(sentence)
  if(doc._.has_coref):       #check if any coreference has been resolved in the doc
    changed_doc = doc._.coref_resolved
    original_sentence.append(sentence)
    changed_sentence.append(changed_doc)
    count = count + 1
    return changed_doc
  return sentence

In [27]:
mao_df["Sentence3"] = mao_df['Sentence2'].apply(coreferenceResolution)
mao_df

,Sentence,NumWords,Sentence2,Sentence3
0,THE IMPORTANCE OF THE PEASANT PROBLEM,6,the importance of the peasant problem,the importance of the peasant problem
1,During my recent visit to Hunan [1] I made a f...,25,during my recent visit to hunan 1 i made a fir...,during my recent visit to hunan 1 i made a fir...
2,In the thirty-two days from January 4 to Febr...,48,in the thirty two days from january to februar...,in the thirty two days from january to februar...
3,Many of the hows and whys of the peasant move...,24,many of the hows and whys of the peasant movem...,many of the hows and whys of the peasant movem...
4,I saw and heard of many strange things of whi...,15,i saw and heard of many strange things of whic...,i saw and heard of many strange things of whic...
...,...,...,...,...
1167,THE PROBLEM OF MILITARY BASES,5,the problem of military bases,the problem of military bases
1168,"The Party in the border area has another task,...",22,the party in the border area has another task ...,the party in the border area has another task ...
1169,The Five Wells mountain area at the juncture ...,88,the five wells mountain area at the juncture o...,the five wells mountain area at the juncture o...
1170,"The way to consolidate these bases is, first,...",27,the way to consolidate these bases is first to...,the way to consolidate these bases is first to...


We have 1172 sentences in our dataset, and 410 sentences has been changed by neuralcoref. Around 410/1172 = 35%

In [28]:
count

410

In [29]:
for i in range(len(original_sentence)):
  print('Orignal: ' + original_sentence[i])
  print('Changed: ' + changed_sentence[i])
  print()

Orignal: in the thirty two days from january to february i called together fact finding conferences in villages and county towns which were attended by experienced peasants and by comrades working in the peasant movement and i listened attentively to their reports and collected a great deal of material
Changed: in the thirty two days from january to february i called together fact finding conferences in villages and county towns which were attended by experienced peasants and by comrades working in the peasant movement and i listened attentively to conferences in villages and county towns which were attended by experienced peasants and by comrades working in the peasant movement reports and collected a great deal of material

Orignal: in a very short time in china s central southern and northern provinces several hundred million peasants will rise like a mighty storm like a hurricane a force so swift and violent that no power however great will be able to hold it back
Changed: in a ver